## AutoLamella Headless - Odemis

In [1]:
# requirements
# stage_position: FibsemStagePosition (stage-bare)                  -> acquire once position selected (can transform from fm-position)
# microcope_state: dict microscope state                            -> acquire once position selected
# reference_image: FibsemImage / OdemisImage (saved to disk)        -> acquire once position selected
# protocol: dict milling protocol for each lamella                  -> load from file, allow user to edit
# config: workflow configuration (supervision, alignment, etc)      -> load from file, allow user to edit

# supported options
# use_microexpansion:

# not supported (for now)
# use_fiducial: # only support non-fiducial milling for now

# odemis: save_milling_position:
# - stage_position:     cryo feature position, defined at milling position: posture_positions[MILLING]
# - microscope_state:   can be extracted from odemis image
# - reference_image:    path to reference image
# - milling_protocol:   dict with milling parameters
#   - patterns:         dict with milling pattern protocol
#   - point:            point in odemis image representing the milling position (generated from correlation)

# req milling parameters
# center -> from correlation / selection
# width  (width of trench)
# height (distance between trenches)
# trench_height (height of trench)
# depth (depth of trench)
# cross_section (cross section of trench Rectangle or CleaningCrossSection)
# milling_current, hfw (all)
# To simplify initially, only use Rough Mill / Polishing

# simplification: 
# all centers are the same
# all hfw are the same

# rough_mill:
# - center: point in odemis image
# - width: 
# - height:
# - trench_height:
# - depth:
# - cross_section: Rectangle

# polishing:
# - center: point in odemis image
# - width:
# - height:
# - trench_height:
# - depth:
# - cross_section: CleaningCrossSection

# microexpansion:
# - center: point in odemis image
# - width:
# - height:
# - distance: distance between expansion trenches
# - depth:

# lamella are created at the ReadyLamella State (after setup in autolamella)
# then the workflow can be run without any user input


In [ ]:
%load_ext autoreload
%autoreload 2

import logging
from pprint import pprint
from fibsem import utils
from autolamella.protocol.validation import validate_protocol
from autolamella.workflows.runners import run_autolamella
from autolamella.compat.odemis import add_odemis_path, add_features_to_experiment, create_experiment_from_odemis

add_odemis_path() # only required when using separate env for openfibsem
from odemis import model
from odemis.acq.feature import CryoFeature, model, read_features
from odemis.acq.move import MicroscopePostureManager, POSITION_NAMES, SEM_IMAGING, MILLING, FM_IMAGING
from odemis.util.filename import make_unique_name

logging.basicConfig(level=logging.INFO)

In [ ]:
features = read_features("/home/patrick/Pictures/my-project-11")
pm = MicroscopePostureManager(model.getMicroscope())

for f in features:
    posture = pm.getCurrentPostureLabel(f.stage_position.value)
    
    sem_pos = pm.to_posture(f.stage_position.value, posture=SEM_IMAGING)
    
    mill_pos  = sem_pos.copy()
    mill_pos.update(pm.stage.getMetadata()[model.MD_FAV_MILL_POS_ACTIVE])
    
    f.posture_positions[FM_IMAGING] = f.stage_position.value
    f.posture_positions[SEM_IMAGING] = sem_pos
    f.posture_positions[MILLING] = mill_pos


In [3]:
# dummy feature creation from odemis
stage_positions = [
    {"x": 0, "y": 0, "z": 0, "rx": 0.31, "rz": 0},
    {"x": 50e-6, "y": 50e-6, "z": 0, "rx": 0.31, "rz": 0}
]

features = []
for stage_position in stage_positions:
    name = make_unique_name(f"Feature-1", [f.name.value for f in features])
    feature = CryoFeature(name=name, stage_position=stage_position, fm_focus_position={"z": 0.169}, posture=MILLING)
    feature.posture_positions[MILLING] = stage_position
    features.append(feature)

logging.info(f"Created {len(features)} features")

In [ ]:
# configuration files
CONFIGURATION_PATH = "/home/patrick/development/openfibsem/fibsem/fibsem/config/odemis-configuration.yaml"
PROTOCOL_PATH = "/home/patrick/development/openfibsem/autolamella/autolamella/protocol/protocol-odemis-on-grid.yaml"
ODEMIS_PROJECT_PATH = "/home/patrick/Pictures/odemis-auto-milling-01"

# connect to microscope
microscope, settings = utils.setup_session(config_path=CONFIGURATION_PATH, 
                                           protocol_path=PROTOCOL_PATH)

# create experiment from odemis project
protocol = validate_protocol(settings.protocol)
experiment = create_experiment_from_odemis(ODEMIS_PROJECT_PATH, protocol=protocol)

# add features to experiment
experiment = add_features_to_experiment(
                                        experiment=experiment, 
                                        features=features,
                                        protocol=settings.protocol
                                        )

logging.info(f"Updated Experiment: {experiment.name}, total: {len(experiment.positions)}")

In [ ]:
from autolamella.structures import AutoLamellaStage
stages_to_complete = [AutoLamellaStage.MillRoughCut]

# run autolamella
experiment = run_autolamella(microscope=microscope, 
                settings=settings, 
                experiment=experiment,
                parent_ui=None, 
                stages_to_complete=stages_to_complete)

# TODO: 
# update features after milling -> status update

In [ ]:
from fibsem.util.filename import get_unique_filename
filenames = [ "test-image.ome.tiff", "test-image.tiff", "test-image.tif"]

# only support two extensions for now (ome.tiff, tiff)
for filename in filenames:
    
    filename = get_unique_filename(filename)
    print(filename)